In [1]:
# Import modules
import torch

In [2]:
# Create a parallel db generator which removes one record from the original db and keeps the rest of the records
def par_db_generator(db, idx):
    return torch.cat((db[0:idx], db[idx+1:]))

In [3]:
# Backle up the parallel databases together
# it depends entirely on the length of the first db
def par_dbs(db):
    """This creates an array with the datase columns = len(db-1)
    Since we are removing one record at a time and returning the remaining values
    """
    db_all = []
    
    for i in range(len(db)):
        par_db = par_db_generator(db, i)
        db_all.append(par_db)
        
    return db_all

In [4]:
# Create a function to generate the required data based on a given sample of users or records
def generate_db(rec_no):
    """This function generates the db that is then usd to generate the parallel dbs"""
    db = torch.rand(rec_no)>0.5
    # print('Original DB\n{}'.format(db))
    
    #Generate the entire parallel db
    pdbs = par_dbs(db)
    
    return db, pdbs    

In [5]:
def db_query(db):
    return db.float().mean()

In [6]:
# Query all the other paraleel DBs and compare with the full database query
def sensitivity(query, num_entries=1000):
    """
    This function takes the query function as an argument and the number of entries
    It initializes the databse and the parallel databases and then performs query for both
    It then uses the qery result to eveluate the similarity
    """
    
    # Generate the data
    torch.manual_seed(500)
    db, pdbs = generate_db(num_entries)
    
    ful_db_query = query(db)
    max_dist = 0
    for pdb in pdbs:
        pdb_query = query(pdb)
        
        # Creating an L1 Sensitivity
        
        db_distance = torch.abs(pdb_query - ful_db_query)

        if db_distance > max_dist:
            max_dist = db_distance
    return max_dist

In [7]:
print(sensitivity(db_query, 2000))

tensor(0.0003)
